In [ ]:
!pip install torch
!pip install transformers
!pip install datasets
!pip install tqdm

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
# old one
# !pip install -q -U transformers
# !pip install -q -U accelerate
# !pip install -q -U bitsandbytes
# !pip install pandas
# !pip install datasets
# conda create -n torch_env transformers accelerate bitsandbytes pytorch
# pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
# conda create -n torch_env transformers pytorch pip

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import torch
from transformers import pipeline, GenerationConfig, AutoModelForSequenceClassification

model_path_statedict = '/content/drive/MyDrive/zephyr_3b_statedict'
# torch.save(model.state_dict(), model_path_statedict)
# print("Saved PyTorch Model State to model.pth")

# load models
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(model_path_statedict))

# save fine tuned_model?
# model_path = '/content/drive/MyDrive/zephyr_3b_model'
# model.save_pretrained(model_path)
base_model = 'stabilityai/stablelm-base-alpha-3b'
# model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels=2)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/708 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/21.1k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/10.2G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of GPTNeoXForSequenceClassification were not initialized from the model checkpoint at stabilityai/stablelm-base-alpha-3b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPTNeoXForSequenceClassification(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50688, 4096)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-15): 16 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=40

In [ ]:
import pandas as pd
from datasets import Dataset

df = pd.read_json('/content/drive/MyDrive/val.model-aware.v2.json')
df["labels"] = df["label"].map({"Hallucination":1, "Not Hallucination":0})
df = df.dropna(how="any", axis=1)
df = df[["hyp","model","labels"]]
df.head()


,hyp,model,labels
0,A sloping top .,ltg/flan-t5-definition-en-base,1
1,To react too much .,ltg/flan-t5-definition-en-base,0
2,The process of spoiling ; the state of being s...,ltg/flan-t5-definition-en-base,1
3,To arrange in a particular way .,ltg/flan-t5-definition-en-base,1
4,A feeling of concern ; a feeling of anxiety .,ltg/flan-t5-definition-en-base,1


In [ ]:
def add_prompt_and_features(example):
    prompt = "Classify the following text as either hallucinating or not hallucinating: "
    example['input_text'] = prompt + example['hyp'] + " Model: " + str(example['model'])
    return example

dataset = Dataset.from_pandas(df)
dataset = dataset.map(add_prompt_and_features)


Map:   0%|          | 0/501 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['hyp', 'model', 'labels', 'input_text'],
    num_rows: 501
})

In [ ]:
from transformers import AutoTokenizer
# base_model = "HuggingFaceH4/zephyr-7b-beta"
# base_model = 'stabilityai/stablelm-zephyr-3b'

tokenizer = AutoTokenizer.from_pretrained(base_model)

# def tokenize_function(examples):
#     return tokenizer(examples['input_text'], padding='max_length', truncation=True, max_length=100)

# tokenized_datasets = dataset.map(tokenize_function, batched=True)
# tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

# added return_tensors='pt' to tokenizer() method to solve error, AttributeError: 'list' object has no attribute 'size'
# actually i added tokenized_datasets.set_format('pt') to solve it instead
def tokenize_function(example):
    return tokenizer(example['input_text'], truncation=True, padding='max_length', max_length=512)

tokenizer.pad_token = tokenizer.eos_token
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['hyp','model','input_text'])
tokenized_datasets.set_format('pt')
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)




tokenizer_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/501 [00:00<?, ? examples/s]

In [ ]:
# inputs = tokenizer(text, return_tensors="pt")
# inputs = {key: value.to(device) for key, value in inputs.items()}

# Perform inference
# with torch.no_grad():
#     outputs = model(**inputs)

# Process outputs (e.g., get logits and predictions)
# logits = outputs.logits
# predictions = torch.argmax(logits, dim=-1)

# print(f"Predictions: {predictions}")

In [ ]:

# train_dataloader = DataLoader(tokenized_datasets['train'], shuffle=True, batch_size=8, collate_fn=data_collator)
# eval_dataloader = DataLoader(tokenized_datasets['test'], batch_size=8, collate_fn=data_collator)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets, batch_size=1, shuffle=True)


In [ ]:
device

'cpu'

In [ ]:
import os
import torch

def save_checkpoint(model, optimizer, epoch, loss, path):
    if not os.path.exists(path):
        os.makedirs(path)
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(checkpoint, os.path.join(path, f'checkpoint_epoch_{epoch}.pth'))
    print(f'Checkpoint saved at epoch {epoch}')

def load_checkpoint(path, model, optimizer):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f'Checkpoint loaded from epoch {epoch}')
    return epoch, loss


In [ ]:
# # chatgpt code
# from transformers import AdamW, get_scheduler
# from tqdm.auto import tqdm

# # Assuming 'model' and 'train_dataloader' are defined
# optimizer = AdamW(model.parameters(), lr=5e-5)
# num_epochs = 5
# num_training_steps = num_epochs * len(train_dataloader)
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=0,
#     num_training_steps=num_training_steps
# )

# # Path to save checkpoints
# checkpoint_path = '/content/drive/MyDrive/your_checkpoint_directory'

# # Start training
# model.train()
# start_epoch = 0  # Change this if resuming from a specific epoch

# # Uncomment this if resuming from a checkpoint
# # start_epoch, _ = load_checkpoint(os.path.join(checkpoint_path, 'checkpoint_epoch_3.pth'), model, optimizer)

# for epoch in range(start_epoch, num_epochs):
#     progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
#     total_loss = 0
#     for step, batch in enumerate(progress_bar):
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**batch)
#         loss = outputs.loss
#         loss.backward()

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()

#         total_loss += loss.item()
#         progress_bar.set_postfix({"loss": loss.item()})

#     # Save checkpoint at the end of each epoch
#     save_checkpoint(model, optimizer, epoch, total_loss / len(train_dataloader), checkpoint_path)


In [ ]:
from transformers import AdamW, get_scheduler
from tqdm.auto import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# model.train()
# for epoch in range(num_epochs):
#     progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
#     for batch in progress_bar:
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**batch)
#         loss = outputs.loss
#         loss.backward()

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.set_postfix({"loss": loss.item()})


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Reduce batch size and use gradient accumulation
batch_size = 1  # Smaller batch size
accumulation_steps = 2  # Accumulate gradients over 8 steps
# Path to save checkpoints
checkpoint_path = '/content/drive/MyDrive/zephyr_3b_checkpoint'

# Start training
model.train()
start_epoch = 1  # Change this if resuming from a specific epoch

# Uncomment this if resuming from a checkpoint
# start_epoch, _ = load_checkpoint(os.path.join(checkpoint_path, 'checkpoint_epoch_3.pth'), model, optimizer)

for epoch in range(num_epochs):
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
    total_loss = 0
    for step, batch in enumerate(progress_bar):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        # Gradient accumulation
        if (step + 1) % accumulation_steps == 0:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        progress_bar.set_postfix({"loss": loss.item()})
        total_loss += loss.item()

    # Perform a final optimization step if needed
    if (step + 1) % accumulation_steps != 0:
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    # Save checkpoint at the end of each epoch
    save_checkpoint(model, optimizer, epoch, total_loss / len(train_dataloader), checkpoint_path)

Epoch 1/3:   0%|          | 0/501 [00:00<?, ?it/s]

In [ ]:
model_path_statedict = '/content/drive/MyDrive/zephyr_3b_statedict
torch.save(model.state_dict(), model_path_statedict)
print("Saved PyTorch Model State to model.pth")

# load models
# model = NeuralNetwork().to(device)
# model.load_state_dict(torch.load("model.pth"))

# save fine tuned_model?
model_path = '/content/drive/MyDrive/zephyr_3b_model'
model.save_pretrained(model_path)

In [ ]:
from sklearn.metrics import accuracy_score

model.eval()
eval_dataloader = DataLoader(tokenized_datasets, batch_size=8)

predictions, true_labels = [], []
with torch.no_grad():
    for batch in eval_dataloader:
        outputs = model(**batch)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=-1).tolist())
        true_labels.extend(batch['labels'].tolist())

accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')


In [6]:
import pathlib
import pandas as pd
file_path = r"/content/drive"
file_path_uni = pathlib.Path(file_path).as_posix()
print(file_path_uni)
# Save the DataFrame as a CSV file at the specified location
pd.DataFrame(true_labels).to_csv(file_path_uni + 'zephyr_true_labels.csv', index=False)

/content/drive
